<a href="https://colab.research.google.com/github/Trupti2795/TransferLearning_InceptionV3/blob/main/InceptionV3_TomatoLeafDisease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

In [5]:
!pip install tensorflow-gpu

     |████████████████████████████████| 394.7MB 45kB/s 


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# New Section

In [3]:
!nvidia-smi

Wed Jan 20 16:26:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:

# import the libraries as shown below
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [7]:
Image_size= [224,224]

Train_Path ='/content/drive/MyDrive/train'
Valid_Path = '/content/drive/MyDrive/valid'

Import the InceptionV3 ,using Imagenet_weights

In [14]:
inception = InceptionV3(input_shape=Image_size + [3], weights='imagenet', include_top=False)

87916544/87910968 [==============================] - 2s 0us/step


In [15]:
#To make use of all weights we do not train existing weights
for layer in inception.layers:
  layer.inception =False 


In [16]:
folders = glob('/content/drive/MyDrive/train*')

In [18]:
X =Flatten()(inception.output)

In [20]:
prediction = Dense(len(folders),activation ='softmax')(X)# concatenating with flatten layer of previous inception

In [25]:
#create model object
model =Model(inputs=inception.input, outputs= prediction)

In [26]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [38]:

# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [39]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [56]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/train',
                                                 target_size = (512,512),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')

Found 330 images belonging to 10 classes.


In [57]:

test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/valid',
                                            target_size = (512,512),
                                            batch_size = 16,
                                            class_mode = 'categorical')

Found 4595 images belonging to 10 classes.


In [61]:

# fit the mod
# # Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

In [ ]:

import matplotlib.pyplot as plt

In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:

# save it as a h5 file


from tensorflow.keras.models import load_model

model.save('model_inception.h5')

In [ ]:
y_pred = model.predict(test_set)

In [ ]:

import numpy as np
y_pred = np.argmax(y_pred, axis=1)

In [ ]:

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [ ]:
model=load_model('model_resnet50.h5')

In [ ]:

img=image.load_img('/content/drive/MyDrive/valid/Coffee/download (2).jpg',target_size=(224,224))

In [ ]:

x=image.img_to_array(img)
x

In [ ]:
x=x/255

In [ ]:

import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

In [ ]:
model.predict(img_data)

In [ ]:

a=np.argmax(model.predict(img_data), axis=1)